In [1]:
from IPython.display import display, HTML
import json

In [2]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

# Simple HTML Embedding

In [3]:
display(HTML("""
<html>
<body>
<div style="width:50px; height: 50px; background-color:#FF0000"/>
</body>
</html>
"""))

# Calling an external library

In [4]:
with open('library.js', 'r') as f:
    lib = f.read()
my_html = """
<html>
    <head>
        <script>{library}</script>
    </head>
    <body>
        <div id="div_id"/>
    </body>
    <script>
        circle();
    </script>
</html>
""".format(library=lib)

display(HTML(my_html))

# Sending data from Python to JS

In [5]:
with open('library.js', 'r') as f:
    lib = f.read()

html_str = """
<html>
<head>
</head>
<body>
<div id="div_data1"/>
</body>
<script type="text/javascript">
{library}
bar_chart("#div_data1", {data})
</script>
</html>
""".format(library=lib, data=json.dumps([0,1,2,3,4]))
display(HTML(html_str))

# Sending data from JS to Python

In [6]:
def target_func(comm, open_msg):
    # comm is the kernel Comm instance

    # Register handler for later messages
    @comm.on_msg
    def _recv(msg):
        # Use msg['content']['data'] for the data in the message
        n = msg['content']['data']['n']
        comm.send({'array': list(range(n))})

get_ipython().kernel.comm_manager.register_target('my_comm_target', target_func)

In [7]:
from string import Template

with open('library.js', 'r') as f:
    lib = f.read()

template = Template("""
<html>
<body>
<div id="div_receive_data"/>
</body>
<script>
$library
function main(){
    let comm = Jupyter.notebook.kernel.comm_manager.new_comm('my_comm_target')
    // Send data
    comm.send({'n': 3})

    // Register a handler
    comm.on_msg(function(msg) {
        let data = msg.content.data.array;
        bar_chart("#div_receive_data", data)
    });
}
main()
</script>
</html>
""")
my_html = template.safe_substitute(library=lib)
display(HTML(my_html))

Please see https://jupyter-notebook.readthedocs.io/en/stable/comms.html for more details